In [11]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 13, Finished, Available)

In [12]:
%%sql
create table if not exists TMDW_FinalProj.gld_product
(
    PID	Long	,
Product_Category string	,
Product string	,
Created_TS	timestamp,
Modified_TS timestamp
)
using delta

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 14, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
Maximum_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from TMDW_FinalProj.gld_product").first()[0]

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 15, Finished, Available)

In [15]:
dataframe_bze=spark.sql("select distinct Product_category, Product from TMDW_FinalProj.bronzesales where Modified_TS>'{}'".format(Maximum_Date))

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 17, Finished, Available)

In [16]:
Maximum_ID=spark.sql("select coalesce(max(PID),0) from TMDW_FinalProj.gld_product").first()[0]

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 18, Finished, Available)

In [17]:
dataframe_final=dataframe_bze.withColumn("PID",monotonically_increasing_id()+Maximum_ID+1)
dataframe_final.createOrReplaceTempView("View_Product")

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 19, Finished, Available)

In [18]:
%%sql
merge into TMDW_FinalProj.gld_product as gp
using View_Product as vp  
on gp.PID=vp.PID
AND gp.Product_Category=vp.Product_Category
when MATCHED then 
update set
gp.Modified_TS= current_timestamp()

when not matched then
INSERT
(
gp.Product,
gp.PID,
gp.Product_Category,
gp.Created_TS,
gp.Modified_TS
)
VALUES
(
vp.Product,
vp.PID,
vp.Product_Category,
current_timestamp(),
current_timestamp()
)

StatementMeta(, d093be53-a18c-4b7b-bdf0-9f217c89684e, 20, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>